In [2]:
import pandas as pd
import numpy as np

#функция превращает таблицу, где первый столбец маги, а остальные нормы по тпг в таблицу для загрузки этих норм в нав
#По умолчанию максимальное значение 1, коэф отставания 0
#индекс = номер магазина

def table_tonav(df):
    
    norms_list = df.T.to_numpy().tolist()
    
    tpg_list = df.columns.tolist()
    
#добавим бк/иф к названиям тпг   
    new_tpg_list = ['БК_' + str(tpg) if int(str(tpg)[:4]) >= 1500 else 'ИФ_' + str(tpg) for tpg in tpg_list]
          
    tpg = [x for x in new_tpg_list for i in range(len(df.index.tolist()))]
    shop = df.index.tolist()*len(df.columns.tolist())
    norms = [round(item) for sublist in norms_list for item in sublist]#Все нормы округляются
    data = {'Товарная Группа':tpg,'Норма':norms,'Макс. допустимое кол-во на точке':[1]*len(norms),'коэф. Корр отставания проведения продаж':[0]*len(norms)}
    new_df = pd.DataFrame(index=shop, data=data)
    new_df.index.name = 'Код'                    
    return new_df

def priority_tonav(df):
    
#индекс = номер магазина    
    norms_list = df.T.to_numpy().tolist()
    tpg_list = df.columns.tolist()
    tpg = [x for x in tpg_list for i in range(len(df.index.tolist()))]
    shop = df.index.tolist()*len(df.columns.tolist())
    norms = [item for sublist in norms_list for item in sublist]#Все нормы округляются
    data = {'Товарное направление':tpg,'Приоритет':[100]*len(norms),'Повышающий коэффициент':[1]*len(norms),'Исключить из распределения':norms}
    new_df = pd.DataFrame(index=shop, data=data)
    new_df.index.name = 'Код'                    
    return new_df

In [2]:
# любая табла с нормами по тпг
df = pd.read_excel(r'C:\Users\Dotsenko.Semen\Documents\145.xlsx',index_col=0)
table_tonav(df).to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\в_нав.xlsx')

In [20]:
# ОБРУЧИ

#загружаем нормы по размерам и по тпг
wed_rings1 = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\матрица обручей.xlsx',
    sheet_name='гладкая_плоская размеры_тпг',index_col=0,skiprows=3)
del wed_rings1['адрес']
del wed_rings1['норма']

ring_cols = [0, 3, 8] + list(range(12,25))
wed_rings2 = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\матрица обручей.xlsx',
    sheet_name='Матрица по тпг',index_col=0,skiprows=3, usecols=ring_cols)

#применяем функцию исправляем максимумы соединяем в один датафрейм и сохраняем
wed_rings_tonav1 = table_tonav(wed_rings1)
wed_rings_tonav1['Макс. допустимое кол-во на точке'] = wed_rings_tonav1['Норма']
wed_rings_tonav2 = table_tonav(wed_rings2)

wed_rings_tonav = pd.concat([wed_rings_tonav1, wed_rings_tonav2 ])

wed_rings_tonav.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\в_нав.xlsx')

In [ ]:
#МУСУЛЬМАНКА
from_26 = [0] + list(range(26,49))
mus_df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\мусульманкэ.xlsx',
    sheet_name='нормы культ',index_col=0,skiprows=2,usecols=from_26)

mus_df.columns = [x[:4] for x in mus_df.columns.tolist()]

mus_df = table_tonav(mus_df)

mus_df.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\мус_внав.xlsx',
sheet_name='Лист1')

In [3]:
# ПО ТГ

#рассчитываем номера столбцов для каждого ТН

if_string_lenth = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='ИФ',nrows=0,skiprows=2)

lst = ['ИФ СЕРЬГИ' if 'ИФ СЕРЬГИ' in str(x) else x for x in if_string_lenth.columns.tolist()]

if_end = len(lst)-lst[::-1].index('ИФ СЕРЬГИ')
if_start = if_end-6
if_columnrange = [1,2] + list(range(if_start,if_end))

bk_string_lenth = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='БК',nrows=0,skiprows=2)

lst = ['БК СЕРЬГИ' if 'БК СЕРЬГИ' in str(x) else x for x in bk_string_lenth.columns.tolist()]

bk_end = len(lst)-lst[::-1].index('БК СЕРЬГИ')
bk_start = bk_end-6
bk_columnrange = [1,2] + list(range(bk_start,bk_end))


# загружаем датафреймы
begin = [1,2]
if_df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='ИФ',skiprows=3,usecols=if_columnrange,index_col=1, names=['блок','СКЛАД',
        'ИФ КОЛЬЦА','ИФ КОЛЬЦА ОБРУЧ','ИФ ПЕЧАТКИ','ИФ ПОДВЕС ДЕКОР','ИФ ПОДВЕС КУЛЬТ','ИФ СЕРЬГИ'])

bk_df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='БК',skiprows=3,usecols=bk_columnrange,index_col=1, names=['блок','СКЛАД',
        'БК КОЛЬЦА','БК КОЛЬЦА ОБРУЧ','БК ПЕЧАТКИ','БК ПОДВЕС ДЕКОР','БК ПОДВЕС КУЛЬТ','БК СЕРЬГИ'])

# убираем закрытые маги
if_df = if_df[(if_df.блок != 'з')]
del if_df['блок']

bk_df = bk_df[(bk_df.блок != 'з')]
del bk_df['блок']

#переворачиваем в нужный формат, соединяем и сохраняем
if_tonav = priority_tonav(if_df)
if_tonav.index.name = 'Код'

bk_tonav = priority_tonav(bk_df)
bk_tonav.index.name = 'Код'

total = if_tonav.append(bk_tonav)

total.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\приоритеты_внав.xlsx',
sheet_name='Лист1')


In [4]:
list(range(12,25))

[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

In [14]:
import pandas as pd
import numpy as np

#загружаем файл
wed_rings = pd.read_excel(r'C:\Users\Dotsenko.Semen\Documents\123.xlsx', index_col='Название из NAV')

In [15]:
wed_rings

,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,Unnamed: 19
Название из NAV,,,,,,,,,,,,,,,,,,,
306,88.000000,42.00000,43,90.000000,22.000000,25.000000,35.000000,82.287078,100.832007,82.251905,48.151746,216.287162,216.216816,54.192697,125.681685,36.843648,65.386484,126.868771,0
569,79.243371,41.55587,43,33.860163,10.000000,18.718476,34.318523,11.628388,14.249062,11.623417,6.804558,30.564592,30.554652,7.658234,17.760691,5.206555,9.240082,17.928444,0
3078,88.000000,42.00000,43,79.874873,22.000000,25.000000,35.000000,47.386323,58.065716,47.366068,27.728949,124.552404,124.511894,31.207727,72.375798,21.217001,37.653847,73.059401,0
3276,88.000000,42.00000,43,55.912411,15.455972,25.000000,35.000000,28.186467,34.538814,28.174419,16.493812,74.086614,74.062518,18.563069,43.050778,12.620357,22.397368,43.457400,0
4389,88.000000,42.00000,43,55.912411,15.455972,25.000000,35.000000,28.186467,34.538814,28.174419,16.493812,74.086614,74.062518,18.563069,43.050778,12.620357,22.397368,43.457400,0


In [44]:
new_df = wed_rings[range(1523,1534)].unstack()

In [45]:
new_df = pd.DataFrame(new_df)

In [46]:
new_df.reset_index(inplace=True)

In [52]:
new_df.round(0)

,level_0,Название из NAV,0
0,1523,306,82.0
1,1523,569,12.0
2,1523,3078,47.0
3,1523,3276,28.0
4,1523,4389,28.0
5,1524,306,101.0
6,1524,569,14.0
7,1524,3078,58.0
8,1524,3276,35.0
9,1524,4389,35.0


In [57]:
a=False
b=True
c=False
if not a or b or not c and a:
    print(1)

1


In [2]:
import pandas as pd
import numpy as np
np.random.seed(35)
a = np.random.randint(1,11,4)
b = np.random.randint(1,21,4)
df = pd.DataFrame({'1516':a,'1517':b,'1518':a })

In [4]:
df1 = df.unstack()

In [10]:
df1


1516  0    10
      1     8
      2     2
      3     1
1517  0     9
      1    12
      2    12
      3     9
1518  0    10
      1     8
      2     2
      3     1
dtype: int32

In [12]:
df1.reset_index().columns

['level_0', 'level_1', 0]